In [2]:
from pypdb import *
from Bio.PDB import PDBParser
import requests as r
from Bio import SeqIO
from io import StringIO
from Bio import pairwise2
from Bio.Seq import Seq
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
from time import sleep
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import os
import OxiAnalysis as OA
import urllib
import sys
import logging
import importlib
importlib.reload(OA)
import warnings; warnings.simplefilter('ignore')

/home/robbe/miniconda3/lib/python3.9/site-packages/pyteomics/auxiliary/patch.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pv) < LooseVersion('0.17'):
/home/robbe/miniconda3/lib/python3.9/site-packages/pyteomics/auxiliary/patch.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pv) < LooseVersion('0.17'):
/home/robbe/.local/lib/python3.9/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/home/robbe/miniconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [3]:
mastermod = pd.read_csv("/home/robbe/ionbot/mastersets/PXD012477_modifications.csv")

In [4]:
mastermod.head()

,Unnamed: 0,protein,uniprot_id,unexpected_modification,position,ionbot_match_id,#PSMs,Accession,spectrum_file,Organism,Cell line,Oxidant treatment,Oxidant dose (µM),Secondary treatment,Secondary treatment dose (µM),Length of oxidant treatment (mins),Length of Secondary treatment (mins),Notes
0,0,1433E_HUMAN,P62258,[4]carbamidomethyl[C],97,0_26870_1,2,12477,1DLC0080815QE_RRAG_001_rep1,Human,Urine,Healthy Ageing,0,None,0,0,0,NaN
1,1,1433E_HUMAN,P62258,[4]carbamidomethyl[C],97,0_26944_1,2,12477,1DLC0080815QE_RRAG_001_rep1,Human,Urine,Healthy Ageing,0,None,0,0,0,NaN
2,2,1433E_HUMAN,P62258,[4]carbamidomethyl[C],98,0_26870_1,1,12477,1DLC0080815QE_RRAG_001_rep1,Human,Urine,Healthy Ageing,0,None,0,0,0,NaN
3,3,1433E_HUMAN,P62258,[4]Carbamidomethyl[D],99,0_26944_1,1,12477,1DLC0080815QE_RRAG_001_rep1,Human,Urine,Healthy Ageing,0,None,0,0,0,NaN
4,4,1433Z_HUMAN,P63104,[4]carbamidomethyl[C],25,0_3262_1,1,12477,1DLC0080815QE_RRAG_001_rep1,Human,Urine,Healthy Ageing,0,None,0,0,0,NaN


In [5]:
mastermod.columns

Index(['Unnamed: 0', 'protein', 'uniprot_id', 'unexpected_modification',
       'position', 'ionbot_match_id', '#PSMs', 'Accession', 'spectrum_file',
       'Organism', 'Cell line', 'Oxidant treatment', 'Oxidant dose (µM)',
       'Secondary treatment', 'Secondary treatment dose (µM)',
       'Length of oxidant treatment (mins)',
       'Length of Secondary treatment (mins)', 'Notes'],
      dtype='object')

In [6]:
master = mastermod.filter(['protein', 'uniprot_id', 'unexpected_modification',
       'position', 'ionbot_match_id', '#PSMs', 'Accession', 'spectrum_file'])

In [7]:
master.head()

,protein,uniprot_id,unexpected_modification,position,ionbot_match_id,#PSMs,Accession,spectrum_file
0,1433E_HUMAN,P62258,[4]carbamidomethyl[C],97,0_26870_1,2,12477,1DLC0080815QE_RRAG_001_rep1
1,1433E_HUMAN,P62258,[4]carbamidomethyl[C],97,0_26944_1,2,12477,1DLC0080815QE_RRAG_001_rep1
2,1433E_HUMAN,P62258,[4]carbamidomethyl[C],98,0_26870_1,1,12477,1DLC0080815QE_RRAG_001_rep1
3,1433E_HUMAN,P62258,[4]Carbamidomethyl[D],99,0_26944_1,1,12477,1DLC0080815QE_RRAG_001_rep1
4,1433Z_HUMAN,P63104,[4]carbamidomethyl[C],25,0_3262_1,1,12477,1DLC0080815QE_RRAG_001_rep1


In [8]:
g = master.groupby(["uniprot_id", "unexpected_modification", "position"])["#PSMs"].sum().reset_index()

In [9]:
g = g[g['#PSMs'] >10]

In [10]:
g[g["unexpected_modification"].isin(OA.modslist)]

,uniprot_id,unexpected_modification,position,#PSMs
47,A0A075B6H7,[344]Arg->GluSA[R],75,41
64,A0A075B6H7,[35]Oxidation[I],69,12
65,A0A075B6H7,[35]Oxidation[S],73,38
66,A0A075B6H7,[35]Oxidation[T],74,221
67,A0A075B6H7,[35]Oxidation[Y],70,639
...,...,...,...,...
97384,sp|TRFE_HUMAN|,[425]Dioxidation[Y],578,290
97385,sp|TRFE_HUMAN|,[425]Dioxidation[Y],593,509
97386,sp|TRFE_HUMAN|,[425]Dioxidation[Y],669,16
97776,sp|TRFE_HUMAN|,[936]Chlorination[Y],155,14


In [16]:
moddict = dict()
moddf = g[g["unexpected_modification"] == "[6]Carboxymethyl[K]"]
moddf

,uniprot_id,unexpected_modification,position,#PSMs
2661,A0M8Q6,[6]Carboxymethyl[K],80,22
4111,B9A064,[6]Carboxymethyl[K],188,22
5736,O15021,[6]Carboxymethyl[K],2326,15
12412,P01042,[6]Carboxymethyl[K],254,41
12413,P01042,[6]Carboxymethyl[K],262,14
15064,P01833,[6]Carboxymethyl[K],239,11
15931,P01834,[6]Carboxymethyl[K],100,110
20730,P02749,[6]Carboxymethyl[K],52,11
23067,P02760,[6]Carboxymethyl[K],57,16
23070,P02760,[6]Carboxymethyl[K],309,11


In [15]:
#Remove contaminants
moddf = moddf[~moddf["uniprot_id"].str.startswith("sp|")]
protein_list = moddf["uniprot_id"].tolist()
poslist = moddf["position"].tolist()
pro_poslist = []
for i in range(len(protein_list)):
    pro_pos = protein_list[i] + "|" + str(poslist[i])
    pro_poslist.append(pro_pos)

for i, pr in enumerate(pro_poslist):
    #Get protein and position
    protein = pr.split("|")[0]
    pos = int(pr.split("|")[1])
    
    #Get Uniprot sequence for protein
    baseUrl="http://www.uniprot.org/uniprot/"
    currentUrl=baseUrl+protein+".fasta"
    sleep(3)
    response = requests.post(currentUrl)
    cData=''.join(response.text)

    Seq=StringIO(cData)
    pSeq=SeqIO.parse(Seq,'fasta')
    
    for record in pSeq:
        Uniprotseq = record.seq

    if Uniprotseq[pos-1] == "K":
        cheat = 10*"X" + str(Uniprotseq) + 10*"X"
        newpos = pos + 9
        try:
    
            prefix = cheat[newpos-8:newpos]
            suffix = cheat[newpos+1:newpos+9]
            mod_res = cheat[newpos]
            alignment = prefix + mod_res + suffix
            print(protein, newpos, alignment)
        except ValueError:
            continue

A0M8Q6 89 LSLTPEQWKSHRSYSCR
B9A064 197 LSLTPEQWKSHRSYSCQ
O15021 2335 QKLSAVGEKQTLSPKHP
P01042 263 QNCDIYPGKDFVQPPTK
P01042 271 KDFVQPPTKICVGCPRD
P01833 248 KNADLQVLKPEPELVYE
P01834 109 QGLSSPVTKSFNRGECX
P02749 61 GEEITYSCKPGYVSRGG
P02760 66 IGSTCPWLKKIMDRMTV
P02760 318 QLWAFDAVKGKCVLFPY
P02763 147 LAFDVNDEKNWGLSVYA
P02763 157 WGLSVYADKPETTKEQL
P02768 84 VNEVTEFAKTCVADESA
P02768 106 SLHTLFGDKLCTVATLR
P02768 169 HDNEETFLKKYLYEIAR
P02768 266 KAEFAEVSKLVTDLTKV
P02768 356 VCKNYAEAKDVFLGMFL
P02768 411 YAKVFDEFKPLVEEPQN
P02768 422 VEEPQNLIKQNCELFEQ
P07911 365 QLKSLGFDKVFMYLSDS
P0CF74 89 LSLTPEQWKSHRSYSCQ
P0CG04 89 LSLTPEQWKSHRSYSCQ
P0DOX7 216 QGLSSPVTKSFNRGECX
P0DOX8 199 LSLTPEQWKSHRSYSCQ
P0DOY2 52 PGAVTVAWKADSSPVKA
P0DOY2 89 LSLTPEQWKSHRSYSCQ
P0DOY3 52 PGAVTVAWKADSSPVKA
P13987 72 KAGLQVYNKCWKFEHCN
P19652 147 FGSYLDDEKNWGLSFYA
Q9UBG3 56 QEFADVIVKPHDPATVD


In [ ]:
# for i in pro_poslist:
cID= 'P01023'                  #i.split("|")[0]

baseUrl="http://www.uniprot.org/uniprot/"
currentUrl=baseUrl+cID+".fasta"
response = r.post(currentUrl)
cData=''.join(response.text)

Seq=StringIO(cData)
pSeq=SeqIO.parse(Seq,'fasta')
pdb_found = Query(cID).search()[0]
for record in pSeq:
    #Uniprotseq = record.seq
    print(record.seq)

In [115]:
g['Modification'] = g['modifications'].apply(OA.get_modification)

In [116]:
gexplode = g.explode("Modification")

In [117]:
gexplode

,matched_peptide,modifications,scan,Modification
0,AAAAAAAAAAAAAAAGAGAGAK,0|[4]Carbamidomethyl[N-TERM],17718,[4]Carbamidomethyl[N-TERM]
1,AAAAAAAAAAAAAAAGAGAGAK,0|[58]Propionyl[N-TERM],21662,[58]Propionyl[N-TERM]
2,AAAAAAAAAAAAAAAGAGAGAK,23|[30]Cation:Na[C-TERM],70689,[30]Cation:Na[C-TERM]
3,AAAAAAAAAAAAAAAGAGAGAK,x|[931]Ethyl+Deamidated[N],21164,[931]Ethyl+Deamidated[N]
4,AAAAAAAAAAAAAAAGAGANAK,20|Gly->Asn[G],17454,Gly->Asn[G]
...,...,...,...,...
81539,YYVTIIDAPNHR,10|Gly->Asn[G],21375,Gly->Asn[G]
81540,YYYAVYDMVVR,10|[35]Oxidation[V],18652,[35]Oxidation[V]
81541,YYYAVYDMVVR,8|[35]oxidation[M],56993,[35]oxidation[M]
81542,YYYAVYDMVVR,9|[35]Oxidation[V],18637,[35]Oxidation[V]


In [118]:
gexplode = gexplode[gexplode["Modification"].isin(OA.modslist)]
gexplode['Position'] = gexplode['modifications'].apply(OA.get_positions)

In [175]:
sub = gexplode[gexplode["Modification"] == "[6]Carboxymethyl[K]"]
sub = sub[sub["modifications"].str.count("\|") == 1]

In [176]:
sub

,matched_peptide,modifications,scan,Modification,Position
1791,AEAESIYQSKYEEIQITAGR,10|[6]Carboxymethyl[K],53013,[6]Carboxymethyl[K],10
2008,AEFVEVTKIVTDITK,8|[6]Carboxymethyl[K],142503,[6]Carboxymethyl[K],8
2797,AFVDFISDEIKEER,11|[6]Carboxymethyl[K],143484,[6]Carboxymethyl[K],11
2981,AGAVEKGVPIYR,6|[6]Carboxymethyl[K],24271,[6]Carboxymethyl[K],6
3667,AIAAPAAEEKEEAR,10|[6]Carboxymethyl[K],7796,[6]Carboxymethyl[K],10
...,...,...,...,...,...
78560,VWAEPCIIDAAKEEYNGVIEEFIATGEK,12|[6]Carboxymethyl[K],49618,[6]Carboxymethyl[K],12
78905,WGTITDCVVMRDPQTKR,16|[6]Carboxymethyl[K],32068,[6]Carboxymethyl[K],16
80271,YHTVNGHNCEVRK,13|[6]Carboxymethyl[K],2502,[6]Carboxymethyl[K],13
80617,YISPDQIADIYKSFIK,12|[6]Carboxymethyl[K],282670,[6]Carboxymethyl[K],12


In [180]:
peptides = sub["matched_peptide"]
position = sub["Position"].reset_index(drop=True)

In [155]:
os.chdir('/home/robbe/ionbot/SE_for_IceLogo')

In [158]:
for mod in OA.modslist:
    with open("{}_ICE.txt".format(mod), "w") as fn:
        sub = gexplode[gexplode["Modification"] == mod]
        sub = sub[sub["modifications"].str.count("\|") == 1]
        peptides = sub["matched_peptide"]
        position = sub["Position"].reset_index(drop=True)
        for i, peptide in enumerate(peptides):
            cheat = 10*"X" + peptide + 10*"X" #Local alignment of peptide in 
            try:
                pos = int(position[i]) + 9
                prefix = cheat[pos-8:pos]
                suffix = cheat[pos+1:pos+9]
                mod_res = cheat[pos]
                alignment = prefix + mod_res + suffix
                fn.write(alignment + "\n")
            except ValueError:
                continue